In [1]:
import numpy as np
import tensorflow as tf
import random
import math
import h5py
import gc
import sys
import pickle

In [2]:
def acc(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 2).T == labels) / predictions.shape[1] / predictions.shape[0])

In [3]:
def mean(numbers):
    return float(sum(numbers)) / max(len(numbers), 1)

In [4]:
hdf_file = 'datasets/pickles/SVHN_multi_box.hdf5'

hdf = h5py.File(hdf_file,'r')
train_dataset = hdf['train_images'][:]
train_labels = hdf['train_labels'][:]
test_dataset = hdf['test_images'][:]
test_labels = hdf['test_labels'][:]
valid_dataset = hdf['valid_images'][:]
valid_labels = hdf['valid_labels'][:]
            
hdf.close()    

print('Training set', train_dataset.shape, train_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)

Training set (229087, 32, 96, 1) (229087, 6)
Test set (13068, 32, 96, 1) (13068, 6)
Validation set (6666, 32, 96, 1) (6666, 6)


In [5]:
train_dataset = train_dataset.astype(np.float32)
test_dataset = test_dataset.astype(np.float32)
valid_dataset = valid_dataset.astype(np.float32)

train_labels = train_labels.astype(np.int32)
test_labels = test_labels.astype(np.int32)
valid_labels = valid_labels.astype(np.int32)

In [6]:
model_to_save = "saved_models/combined/box_on_mnist/CNN_SVHN_Box_on_Mnist.ckpt"
saved_mnist_model = "saved_models/mnist/CNN_SVHN_Mnist.ckpt"

In [7]:
graph_svhn = tf.Graph()

with graph_svhn.as_default():
    HEIGHT = 32
    WIDTH = 32*3

    X = tf.placeholder(tf.float32, [None, HEIGHT, WIDTH, 1])
    Y_ = tf.placeholder(tf.int32, [None, 6])
    
    # Learning Rate - alpha
    alpha = tf.placeholder(tf.float32)
    
    # Dropout Probablity
    pkeep = tf.placeholder(tf.float32)
    
    # 6 Layers and their no of neurons
    # 3 Convolutional Layers and a fully connected layer
    K = 12     # First Conv Layer with depth 12
    L = 24     # Second Conv Layer with depth 24
    M = 36    # Third Conv layer with depth 36
    N = 300   # Fourth Fully Connected layer with 300 neurons
    P = 200   # Fifth Fully Connected layer with 200 neurons
    # Last one will be softmax layer with 10 output channels
    
    W1 = tf.Variable(tf.truncated_normal([6, 6, 1, K], stddev=0.1), name="W1")    # 6x6 patch, 1 input channel, K output channels
    B1 = tf.Variable(tf.constant(0.1, tf.float32, [K]), name="B1")
    
    W2 = tf.Variable(tf.truncated_normal([5, 5, K, L], stddev=0.1), name="W2")
    B2 = tf.Variable(tf.constant(0.1, tf.float32, [L]), name="B2")
    
    W3 = tf.Variable(tf.truncated_normal([4, 4, L, M], stddev=0.1), name="W3")
    B3 = tf.Variable(tf.constant(0.1, tf.float32, [M]), name="B3")
    
    W5_1 = tf.Variable(tf.truncated_normal([P, 11], stddev=0.1), name="W5_1")
    B5_1 = tf.Variable(tf.constant(0.1, tf.float32, [11]), name="B5_1")
    
    W5_2 = tf.Variable(tf.truncated_normal([P, 11], stddev=0.1), name="W5_2")
    B5_2 = tf.Variable(tf.constant(0.1, tf.float32, [11]), name="B5_2")
    
    W5_3 = tf.Variable(tf.truncated_normal([P, 11], stddev=0.1), name="W5_3")
    B5_3 = tf.Variable(tf.constant(0.1, tf.float32, [11]), name="B5_3")
    
    W5_4 = tf.Variable(tf.truncated_normal([P, 11], stddev=0.1), name="W5_4")
    B5_4 = tf.Variable(tf.constant(0.1, tf.float32, [11]), name="B5_4")
    
    W5_5 = tf.Variable(tf.truncated_normal([P, 11], stddev=0.1), name="W5_5")
    B5_5 = tf.Variable(tf.constant(0.1, tf.float32, [11]), name="B5_5")
    
    # Model
    stride = 1  # output is 32x96
    Y1 = tf.nn.relu(tf.nn.conv2d(X, W1, strides=[1, stride, stride, 1], padding='SAME') + B1)
    
    stride = 2  # output is 16x48
    Y2 = tf.nn.relu(tf.nn.conv2d(Y1, W2, strides=[1, stride, stride, 1], padding='SAME') + B2)
    
    stride = 2  # output is 8x24
    Y3 = tf.nn.relu(tf.nn.conv2d(Y2, W3, strides=[1, stride, stride, 1], padding='SAME') + B3)

    # reshape the output from the third convolution for the fully connected layer
    shape = Y3.get_shape().as_list()
    YY = tf.reshape(Y3, shape=[-1, shape[1] * shape[2] * shape[3]])
    
    W4 = tf.Variable(tf.truncated_normal([shape[1] * shape[2] * shape[3], N], stddev=0.1), name="W4")
    B4 = tf.Variable(tf.constant(0.1, tf.float32, [N]), name="B4")
    
    W5 = tf.Variable(tf.truncated_normal([N, P], stddev=0.1), name="W5")
    B5 = tf.Variable(tf.constant(0.1, tf.float32, [P]), name="B5")

    Y4 = tf.nn.relu(tf.matmul(YY, W4) + B4)
    Y5 = tf.nn.relu(tf.matmul(Y4, W5) + B5)
    
    Y_F = tf.nn.dropout(Y5, pkeep)
    
    Ylogits_1 = tf.matmul(Y_F, W5_1) + B5_1
    Ylogits_2 = tf.matmul(Y_F, W5_2) + B5_2
    Ylogits_3 = tf.matmul(Y_F, W5_3) + B5_3
    Ylogits_4 = tf.matmul(Y_F, W5_4) + B5_4
    Ylogits_5 = tf.matmul(Y_F, W5_5) + B5_5   
    ## ('Ylogits_1 shape : ', [None, 11])
    
    Y_1 = tf.nn.softmax(Ylogits_1)
    Y_2 = tf.nn.softmax(Ylogits_2)
    Y_3 = tf.nn.softmax(Ylogits_3)
    Y_4 = tf.nn.softmax(Ylogits_4)
    Y_5 = tf.nn.softmax(Ylogits_5)
   
    cross_entropy = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(Ylogits_1, Y_[:,1])) +\
    tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(Ylogits_2, Y_[:,2])) +\
    tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(Ylogits_3, Y_[:,3])) +\
    tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(Ylogits_4, Y_[:,4])) +\
    tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(Ylogits_5, Y_[:,5]))

    train_prediction = tf.pack([Y_1, Y_2, Y_3, Y_4, Y_5])
    
    train_step = tf.train.AdamOptimizer(alpha).minimize(cross_entropy)
    
    W_s = tf.pack([tf.reduce_max(tf.abs(W1)),tf.reduce_max(tf.abs(W2)),tf.reduce_max(tf.abs(W3))\
                   ,tf.reduce_max(tf.abs(W4)),tf.reduce_max(tf.abs(W5))])
    b_s = tf.pack([tf.reduce_max(tf.abs(B1)),tf.reduce_max(tf.abs(B2)),tf.reduce_max(tf.abs(B3))\
                   ,tf.reduce_max(tf.abs(B4)),tf.reduce_max(tf.abs(B5))])
    
    model_saver = tf.train.Saver()

In [8]:
train_data = train_dataset
label_data = train_labels
print('train : ', train_data.shape, '  test : ', label_data.shape)

box_train_dict = {}
batch_size = 128
num_steps = int(label_data.shape[0] / batch_size)
num_epochs = 25

with tf.Session(graph=graph_svhn) as session:
    # tf.global_variables_initializer().run()
    model_saver.restore(session, saved_mnist_model)
    print('Initialized')

    test_batch = int(test_dataset.shape[0]/num_epochs)
    test_acc = list()
    
    for epoch in range(num_epochs - 1):
        res_epoch = {}
        t_data = test_dataset[epoch*test_batch:(epoch+1)*test_batch]
        t_label = test_labels[epoch*test_batch:(epoch+1)*test_batch]
        
        for step in range(num_steps - 1):
            max_learning_rate = 0.0005
            min_learning_rate = 0.0001

            decay_speed = 5000.0
            learning_rate = min_learning_rate + (max_learning_rate - min_learning_rate) * math.exp(-step/decay_speed)

            batch_data = train_data[step*batch_size:(step + 1)*batch_size, :, :, :]
            batch_labels = label_data[step*batch_size:(step + 1)*batch_size, :]

            feed_dict = {X : batch_data, Y_ : batch_labels, pkeep : 0.80, alpha : learning_rate}
            _, l, train_pred, W, b = session.run([train_step, cross_entropy, train_prediction, W_s, b_s], feed_dict=feed_dict)
            accuracy = float(acc(train_pred, batch_labels[:,1:6]))

            if (step % 500 == 0):
                minibatch = {}
                minibatch['loss'] = l
                minibatch['W'] = W
                minibatch['B'] = b
                minibatch['accuracy'] = "%.2f" % accuracy

                res_epoch[int(step/500)] = minibatch
                print('Loss at step %d: %f' % (step, l))
                print('Minibatch accuracy: %.1f%%' % acc(train_pred, batch_labels[:,1:6]))
                print('W : ',W)
                print('B : ',b)
                print('    ')
                
        box_train_dict[epoch+1] = res_epoch

        epoch_acc = 0
        for f in res_epoch:
            minibatch = res_epoch[f]
            epoch_acc += float(minibatch['accuracy'])
        epoch_acc = float(epoch_acc/len(res_epoch))
        
        _, l, predictions = session.run([train_step, cross_entropy, train_prediction], feed_dict={X : t_data, Y_ : t_label, pkeep : 1.0, alpha : 0.002})
        accuracy = float(acc(predictions, t_label[:,1:6]))
        test_acc.append(accuracy)

        print('------------------------------------')
        print('Epoch',epoch+1,' Complete with accuracy: %.2f%%' % epoch_acc)
        print('Epoch',epoch+1,' Test Accuracy : %.2f%%' % accuracy)
        print('------------------------------------')
        print('        ')
            
    print('Training Complete on MNIST Data')
    print('Test Accuracy : ', mean(test_acc))
    
    save_path = model_saver.save(session, model_to_save)
    print("Model saved in file: %s" % save_path)

train :  (229087, 32, 96, 1)   test :  (229087, 6)
Initialized
Loss at step 0: 11.220667
Minibatch accuracy: 31.1%
W :  [ 0.30989823  0.38223922  0.43278304  0.77872795  0.57660186]
B :  [ 0.09201913  0.22464228  0.26228482  0.22948168  0.23504254]
    
Loss at step 500: 1.837270
Minibatch accuracy: 88.9%
W :  [ 0.28472903  0.37438309  0.43183544  0.77871025  0.57955348]
B :  [ 0.18997099  0.2468127   0.25204077  0.2332004   0.26845166]
    
Loss at step 1000: 1.992791
Minibatch accuracy: 87.3%
W :  [ 0.27960128  0.40311718  0.43189716  0.77871025  0.57955348]
B :  [ 0.18094735  0.25329784  0.2523236   0.23172502  0.27208018]
    
Loss at step 1500: 1.536280
Minibatch accuracy: 90.3%
W :  [ 0.27532208  0.42615762  0.43135744  0.77871025  0.57955348]
B :  [ 0.17686723  0.2614899   0.25108984  0.22919612  0.27384588]
    
------------------------------------
Epoch 1  Complete with accuracy: 74.41%
Epoch 1  Test Accuracy : 90.73%
------------------------------------
        
Loss at step 

In [11]:
file = 'results/Box_on_Mnist.pickle'

with open(file, 'wb') as handle:
    pickle.dump(box_train_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [12]:
with tf.Session(graph=graph_svhn) as session: 
    print('Initialized')
    batch = 1000
    
    valid_acc = list()
    valid_no = int(valid_labels.shape[0] /  batch)
    for i in range(valid_no - 1):
        model_saver.restore(session, model_to_save)
        data = valid_dataset[i*batch:(i+1)*batch]
        labels = valid_labels[i*batch:(i+1)*batch]
        
        _, l, predictions = session.run([train_step, cross_entropy, train_prediction], feed_dict={X : data, Y_ : labels, pkeep : 1.0, alpha : 0.002})
        accuracy = acc(predictions, labels[:,1:6])
        valid_acc.append(accuracy)
        
        print('Valid-Accuracy', i)
        print('Valid accuracy: ', accuracy)
        print('        ')
            
    valid_avg = mean(valid_acc)
    
    print('-----  FINAL  ------')
    print('Final Validation Set Accuracy : ',"%.2f" % valid_avg)

Initialized
Valid-Accuracy 0
Valid accuracy:  96.3
        
Valid-Accuracy 1
Valid accuracy:  96.12
        
Valid-Accuracy 2
Valid accuracy:  95.56
        
Valid-Accuracy 3
Valid accuracy:  95.84
        
Valid-Accuracy 4
Valid accuracy:  95.52
        
-----  FINAL  ------
Final Validation Set Accuracy :  95.87
